In [19]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import re
import torch

In [2]:
#Load File
#Change the address as appropriate
nep_names = open(r'C:\Users\Mnsh\Documents\data\nepali_names.txt', 'r').read()
nep_names[:100]

'Aabha\nAabriti\nAadya\nAagnya\nAagya\nAahana\nAaisha\nAaishka\nAaja\nAakanchhya\nAakreeti\nAakritee\nAakriti\nAan'

In [3]:
#Preprocessing
nep_names = nep_names.lower()
nep_names = nep_names.replace("prajapati-shanti", "").replace("chaudhary-roshan", "").replace("ac-sah", "").replace("dutta-sanjib","")
nep_names = nep_names.replace(".", "")
nep_names = re.sub("[\(].*?[\)]", "", nep_names)
set(nep_names)

{'\n',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [4]:
#Find the length of longest name and numbers of character in it

this_name = 0

#Count chars in longest name
longest = 0

#current name
x_name =[]
prev_longest = 0
for i in range(len(nep_names)):
        
    this_name += 1
    x_name.append(nep_names[i])
        
    # is not alphabet
    if not nep_names[i].isalpha():
        
        prev_longest = longest
        
        longest = max(this_name, longest)
        if prev_longest < longest:
            
            #the name catches the longest name
            the_name = x_name
            prev_longest = longest
        x_name =[]
        
        this_name = 0

In [9]:
x_name, the_name, longest

(['z', 'i', 'p', 'p', 'y'],
 ['k',
  'e',
  's',
  'h',
  'w',
  'a',
  's',
  'w',
  'a',
  'r',
  'a',
  'n',
  'a',
  'n',
  'd',
  'a',
  '\n'],
 17)

In [11]:
data_size = len(nep_names)
chars = sorted(list(set(nep_names))) #make a list of unique chars and sort them
vocab_size = len(chars)
print("Total chars incl end_line: %d and there are unique chars: %d" % ( data_size, vocab_size))

Total chars incl end_line: 61798 and there are unique chars: 27


In [12]:
#Use a dict to convert from char to index and vice versa
char_to_ix = {ch:i for i, ch in enumerate(chars)}  #has to be resorted despite having a sorted list
ix_to_char = {i:ch for i, ch in enumerate(chars)}

In [20]:
#From name as sequence of chars --> to sequence of ints

#The first one is with padding --> names shorter than selected sequence length will be padded
def get_tensors(names_list, seq_len):
    names_tensor = torch.zeros(len(names_list), seq_len)
    
    
    for i in range(len(names_list)):
        names_in_ints = ([char_to_ix[j] for j in names_list[i]])
        #if len(names_in_ints) < 17:
        k = seq_len - len(names_in_ints)
        fill_0 = [0] * k
        names_in_ints.extend(fill_0)
        names_tensor[i,:] = torch.tensor(names_in_ints)
        
    input_tensor = torch.zeros(names_tensor.shape)
    input_tensor[:, 1:] = names_tensor[:, :-1]
    #input_tensor[:10, :]
        
    return(input_tensor, names_tensor)

#Alternative version --> The whole names are treated as continuous sequence
#0 ('\n') --> works as a marker where one name ends and other begins
#Apparently --> the next name not only starts after 0, but it is related to the one before
def get_tensors_alt(names_list, seq_len):
    #names_tensor = torch.zeros(len(names_list), 17)
    combined_names_ints = []
    whole_input_names = [0]
    
    for i in range(len(names_list)):
        names_tensor = torch.zeros(len(names_list[i]), 1)
        names_in_ints = ([char_to_ix[j] for j in names_list[i]])
        names_in_ints.append(0)
        combined_names_ints.extend(names_in_ints)
    whole_input_names.extend(combined_names_ints[:-1])
    num_rows = len(whole_input_names)//seq_len
    remainder = len(whole_input_names) - num_rows * seq_len
    inputs_new, targets_new = torch.tensor(whole_input_names[:-remainder]).view(-1, seq_len), torch.tensor(combined_names_ints[:-remainder]).view(-1, seq_len) 
    return(inputs_new, targets_new)

In [15]:
nep_names[:10]

'aabha\naabr'

In [16]:
#str_list = list(filter(None, str_list)) 

#regroup in names removin \n
all_names = nep_names.split('\n')

#enmpy names were innterferring --> ''
all_names = list(filter(None, all_names)) 

#sort such that order is maintained
all_names.sort()

len(all_names)

8323

In [23]:
i1,t1 = get_tensors(all_names[-10:], 15)
i1,t1

(tensor([[ 0., 26.,  1.,  8.,  5.,  5., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  1., 11., 11.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  5., 14.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  5., 14.,  9., 19.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  5., 14.,  9., 19.,  8.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  5., 14.,  9., 20.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  5., 14.,  9., 20.,  8.,  1., 18.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  9.,  1., 21., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  9.,  7.,  1., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.],
         [ 0., 26.,  9., 16., 16., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.]]),
 tensor([[26.,  1.,  8.,  5.,  5., 18.

In [39]:
''.join([ix_to_char[k] for k in i1[0].tolist()]), ''.join([ix_to_char[k] for k in t1[0].tolist()] )

('\nzaheer\n\n\n\n\n\n\n\n', 'zaheer\n\n\n\n\n\n\n\n\n')

In [24]:
i2,t2 = get_tensors_alt(all_names[-10:], 15)
i2,t2

(tensor([[ 0, 26,  1,  8,  5,  5, 18,  0, 26,  1, 11, 11,  9,  0, 26],
         [ 5, 14,  9,  0, 26,  5, 14,  9, 19,  8,  0, 26,  5, 14,  9],
         [19,  8,  1,  0, 26,  5, 14,  9, 20,  8,  0, 26,  5, 14,  9],
         [20,  8,  1, 18,  0, 26,  9,  1, 21, 12,  0, 26,  9,  7,  1]]),
 tensor([[26,  1,  8,  5,  5, 18,  0, 26,  1, 11, 11,  9,  0, 26,  5],
         [14,  9,  0, 26,  5, 14,  9, 19,  8,  0, 26,  5, 14,  9, 19],
         [ 8,  1,  0, 26,  5, 14,  9, 20,  8,  0, 26,  5, 14,  9, 20],
         [ 8,  1, 18,  0, 26,  9,  1, 21, 12,  0, 26,  9,  7,  1, 18]]))

In [38]:
''.join([ix_to_char[k] for k in i2[0].tolist()]), ''.join([ix_to_char[k] for k in t2[0].tolist()] )


('\nzaheer\nzakki\nz', 'zaheer\nzakki\nze')

In [31]:
i2[0].tolist()

[0, 26, 1, 8, 5, 5, 18, 0, 26, 1, 11, 11, 9, 0, 26]

In [43]:
#find all names starting from each character
#There are no names starting from 'x'
counter = 1
num_names_each_char = []

for i in range(1, len(all_names)):
    if all_names[i-1][0] == all_names[i][0]:
        counter += 1
    else:
        print(all_names[i-1][0], all_names[i][0])
        num_names_each_char.append(counter)
        counter = 1
num_names_each_char.append(counter) #As the last char ('z) is not yet appended

a b
b c
c d
d e
e f
f g
g h
h i
i j
j k
k l
l m
m n
n o
o p
p q
q r
r s
s t
t u
u v
v w
w y
y z


In [45]:
num_names_each_char, counter, torch.sum(torch.tensor(num_names_each_char)), len(all_names)

([957,
  663,
  163,
  431,
  75,
  25,
  168,
  150,
  86,
  283,
  381,
  152,
  442,
  462,
  44,
  678,
  2,
  701,
  1888,
  210,
  125,
  69,
  10,
  148,
  10],
 10,
 tensor(8323),
 8323)

In [46]:
def softmax(x):      
    e_x= np.exp(x)
    return (e_x/np.sum(e_x))
softmax([2, 3, 4])

array([0.09003057, 0.24472847, 0.66524096])

In [47]:
some_one = np.arange(-10, 10)#.reshape(10, 2)
next_one = np.arange(-3, 30)

def clip_it(it):
    
    it = np.maximum(-5, it)# axis =0)
    it = np.minimum(it, 5)# axis =0)
    return it
print(clip_it(some_one))
some_one

[-5 -5 -5 -5 -5 -5 -4 -3 -2 -1  0  1  2  3  4  5  5  5  5  5]


array([-10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,
         3,   4,   5,   6,   7,   8,   9])

In [85]:
def init_weights_backup(h_size, vocab_size):
    Wxh = np.random.randn(h_size, vocab_size)*0.001
    Whh = np.random.randn(h_size, h_size)*0.001
    Why = np.random.randn(vocab_size, h_size)*0.001
    bh = np.zeros((h_size, 1))
    by = np.zeros((vocab_size, 1))
    return Wxh, Whh, Why, bh, by

#WEIGHT INITIALIZATIO - https://github.com/pangolulu/rnn-from-scratch

def init_weights(h_size, vocab_size):
    #Wxh = torch.randn(h_size, vocab_size)*0.001
    Wxh = torch.FloatTensor(h_size, vocab_size).uniform_(-np.sqrt(1. / vocab_size), np.sqrt(1. / vocab_size))
    #Whh = torch.randn(h_size, h_size)*0.001
    Whh = torch.FloatTensor(h_size, h_size).uniform_(-np.sqrt(1. / h_size), np.sqrt(1. / h_size))

    #Why = torch.randn(vocab_size, h_size)*0.001
    Why = torch.FloatTensor(vocab_size, h_size).uniform_(-np.sqrt(1. / h_size), np.sqrt(1. / h_size))

    bh = torch.zeros((h_size, 1))
    by = torch.zeros((vocab_size, 1))
    return Wxh, Whh, Why, bh, by

In [66]:
h_size =128 #Number nodes in hidden state
#vocab_size = 27

Wxh, Whh, Why, bh, by = init_weights(h_size, vocab_size)

In [67]:
input_seq, target_seq = get_tensors(all_names[:950], 15)

In [63]:
#Forward propagation
#Since the inputs, cell states, outputs are necessary for backprop, we will save them in a dictionary
#As first hidden state expects input previous hidden state as input, we initiate hs[-1] as 0 vector of same dimension
#Inputs and cell states are taken forward as given in the equation
#Loss depends upon the probability assigned to the target character
#Sum losses over each step to compute loss for the whole sequence

# mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
# mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

mWxh, mWhh, mWhy = torch.zeros_like(Wxh), torch.zeros_like(Whh), torch.zeros_like(Why)
mbh, mby = torch.zeros_like(bh), torch.zeros_like(by) # memory variables for Adagrad

l_r = 0.01
eps = 1e-6
losses = []
#lr = 0.001
seq_len = input_seq.size()[1]
loss = 0
xs, hs, ps, ys, yt = {}, {}, {}, {}, {}
hs[-1] = torch.zeros((h_size, 1))
for k in range(100):
    for m in range(input_seq.size()[0]):
        cur_inp_seq = input_seq[m]
        cur_tar_seq = target_seq[m].type(torch.LongTensor)
        loss = 0
        for i in range(seq_len):
        #for i in range(10):
            one_hot = torch.zeros(27, 1)
            one_hot[(cur_inp_seq[i].type(torch.LongTensor))] = 1
            #print(one_hot)
            #print(xs[i])
            xs[i] = one_hot
            hs[i] = torch.tanh(torch.mm(Wxh, xs[i]) + torch.mm(Whh, hs[i-1]) + bh)
            ys[i] = torch.mm(Why, hs[i]) + by
            ps[i] = torch.exp(ys[i]) / torch.sum(torch.exp(ys[i]))
            #target = torch.argmax(targets[i])
            target = cur_tar_seq[i]
            loss += -torch.log(ps[i][target, 0])


            #Backward propagation
        #To get derivatives, initialize matrices of same dimension, all 0s
        dWhy = torch.zeros(Why.shape)
        dby = torch.zeros(by.shape)
        dWhh = torch.zeros(Whh.shape)
        dbh = torch.zeros(bh.shape)
        dWxh = torch.zeros(Wxh.shape)

        #Start working in reverse order and collect derivative at each step
        for i in range(seq_len)[::-1]:
        #for i in range(10)[::-1]:
            

            #position of 1 in target vector (y), implies 100% probability of that character
            #####y_idx = torch.argmax(targets[i])  #position of 1 in target vector (y), implies 100% probability of that character


            y_idx = cur_tar_seq[i]
            #select the probability at same position and subtract 1 (y_hat - y)    
            ps[i][y_idx] -= 1 


            #For Why and by derivative with respect to error is sum of derivatives at each step or step-error
            dWhy_step_i=ps[i] @ (hs[i]).t()      #(y_hat - y) @ hs.T - for each step

            dby = dby + ps[i]                    #ps[i] represents (y_hat - y) as computed above

            dWhy = dWhy + dWhy_step_i            #sum over each step


            #Computing delta for Whx, Whh and bh invloves computing step-error
            #Followed by finding differential of parameters each step-error over the whole sequnce
            #computing and updating delta_z helps to keep track of and sum up throught out the sequence

            #delta with respect to a single step
            delta_recent = (Why.t()@ ps[i]) * (1-hs[i]**2)

            #taking all the way back to first step
            for j in range(i+1)[::-1]:

                dWhh_ij= delta_recent @ hs[j-1].t()
                dbh = dbh + delta_recent

                dWhh = dWhh + dWhh_ij

                dWxh_ij= delta_recent @ xs[j].t()
                dWxh = dWxh + dWxh_ij

                #update delta_z such that it goes back one step all while computing the derivative with repect
                #to same step-error
                delta_recent = (Whh.t() @ delta_recent) * (1-hs[j-1]**2)

    
        
        
        mWhy += dWhy * dWhy
        mWxh += dWxh * dWxh
        mWhh += dWhh * dWhh
        mbh += dbh * dbh
        mby += dby * dby
        
        Why += - l_r * dWhy/np.sqrt(mWhy +eps)
        Whh +=  - l_r * dWhh/np.sqrt(mWhh +eps)
        Wxh +=  - l_r * dWxh/np.sqrt(mWxh +eps)
        by +=  - l_r * dby/np.sqrt(mby +eps)
        bh +=  - l_r * dbh/np.sqrt(mbh +eps)
        
        
#         Why = Why - lr * dWhy
#         Whh = Whh - lr * dWhh
#         Wxh = Wxh - lr * dWxh
#         by = by - lr *dby
#         bh = bh - lr *dbh


    if k % 2 ==0 :
        print(k, loss.item())
        losses.append(loss.item())
    #adjust the parameters


0 9.253104209899902
2 8.845074653625488
4 8.646018028259277
6 8.44072437286377
8 8.257573127746582
10 8.103699684143066
12 7.975302696228027
14 7.866878032684326
16 7.7739481925964355
18 7.6932148933410645
20 7.622260570526123
22 7.559297561645508
24 7.502967834472656
26 7.4522175788879395
28 7.406204700469971
30 7.364236354827881
32 7.325735092163086
34 7.290223121643066
36 7.2573018074035645
38 7.2266316413879395
40 7.197950839996338
42 7.1710381507873535
44 7.1457109451293945
46 7.121829509735107
48 7.099274635314941
50 7.077957630157471
52 7.057797431945801
54 7.0387282371521
56 7.020697116851807
58 7.003650665283203
60 6.987541675567627
62 6.972328186035156
64 6.957968235015869
66 6.944422245025635
68 6.931650638580322
70 6.919619560241699
72 6.908289432525635
74 6.8976240158081055
76 6.887590408325195
78 6.878159523010254
80 6.869293689727783
82 6.8609700202941895
84 6.853157997131348
86 6.845827102661133
88 6.838954925537109
90 6.832515716552734
92 6.826486110687256
94 6.8208384

In [107]:
from numpy.random import choice

def gen_names(Wxh, Whh, Why, bh, by):
    new_names = []
    cur_inp_char = torch.Tensor([0])
    elements = [i for i in range(27)]
    gen_name = []
    for x in range(50):
        gen_name = []
        for p in range(seq_len):
            one_hot = torch.zeros(27, 1)
            #one_hot[(cur_inp_seq[p].type(torch.LongTensor))] = 1
            one_hot[(cur_inp_char.type(torch.LongTensor))] = 1
            #print(one_hot)
            #print(xs[i])
            xs[p] = one_hot
            hs[p] = torch.tanh(torch.mm(Wxh, xs[p]) + torch.mm(Whh, hs[p-1]) + bh)
            ys[p] = torch.mm(Why, hs[p]) + by
            ps[p] = torch.exp(ys[p]) / torch.sum(torch.exp(ys[p]))

            #cur_inp_char = torch.topk(ps[p], 1, dim = 0)[1]

            w=np.array(list(ps[p]))
            #print(choice(elements, p=w/sum(w)))
            cur_inp_char = torch.Tensor([choice(elements, p=w/sum(w))])
            #cur_inp_char = torch.Tensor([np.random.choice(list(torch.topk(ps[p], 2, dim = 0)[1]))])
            #gen_name.append(cur_inp_char.item())
            if cur_inp_char ==0:
                break
            gen_name.append(cur_inp_char.item())
        new_names.append(''.join([ix_to_char[f] for f in gen_name]))
    #print(new_names)
    return(new_names)

In [64]:
gen_names(Wxh, Whh, Why, bh, by)

['avasya',
 'alex',
 'ashwip',
 'anushya',
 'aishan',
 'amakashees',
 'angadres',
 'aygab',
 'aviyat',
 'arjan',
 'ayusha',
 'arisher',
 'anir',
 'anushiy',
 'avinakh',
 'aradh',
 'arutrna',
 'arit',
 'bmees',
 'adilp',
 'asmi',
 'arbind',
 'anudhmal',
 'assh',
 'apgaral',
 'arjan',
 'angesha',
 'arna',
 'asdeet',
 'apca',
 'amja',
 'alna',
 'adeya',
 'asma',
 'avisekh',
 'anushra',
 'amod',
 'amikesh',
 'ashukana',
 'anuv',
 'asmi',
 'ashsit',
 'aray',
 'ajanna',
 'asha',
 'armal',
 'ayushu',
 'asam',
 'awarit',
 '']

In [91]:
#Forward propagation
#Since the inputs, cell states, outputs are necessary for backprop, we will save them in a dictionary
#As first hidden state expects input previous hidden state as input, we initiate hs[-1] as 0 vector of same dimension
#Inputs and cell states are taken forward as given in the equation
#Loss depends upon the probability assigned to the target character
#Sum losses over each step to compute loss for the whole sequence

# mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
# mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

def rnn_net(Wxh, Whh, Why, bh, by, input_seq, target_seq, epochs):
    seq_len = input_seq.size()[1] #+1
    #print(seq_len)
    #losses = []
    mWxh, mWhh, mWhy = torch.zeros_like(Wxh), torch.zeros_like(Whh), torch.zeros_like(Why)
    mbh, mby = torch.zeros_like(bh), torch.zeros_like(by) # memory variables for Adagrad

    eps = 1e-6

    lr = 0.001
    #seq_len = input_tensor.size()[1]
    loss = 0
    xs, hs, ps, ys, yt = {}, {}, {}, {}, {}
    hs[-1] = torch.zeros((h_size, 1))
    for k in range(epochs):
        #hs[-1] = torch.zeros((h_size, 1))
        for m in range(input_seq.size()[0]):
            cur_inp_seq = input_seq[m]#.view(-1,1)
            cur_tar_seq = target_seq[m].type(torch.LongTensor)
            
            loss = 0
            for i in range(seq_len):
            #for i in range(10):
                one_hot = torch.zeros(27, 1)
                
                one_hot[(cur_inp_seq[i].type(torch.LongTensor))] = 1
                
                xs[i] = one_hot
                hs[i] = torch.tanh(torch.mm(Wxh, xs[i]) + torch.mm(Whh, hs[i-1]) + bh)
                ys[i] = torch.mm(Why, hs[i]) + by
                ps[i] = torch.exp(ys[i]) / torch.sum(torch.exp(ys[i]))
                #target = torch.argmax(targets[i])
                target = cur_tar_seq[i]
                loss += -torch.log(ps[i][target, 0])

            ####hs[-1] = hs[i]
            
            #Backward propagation
            #To get derivatives, initialize matrices of same dimension, all 0s
            dWhy = torch.zeros(Why.shape)
            dby = torch.zeros(by.shape)
            dWhh = torch.zeros(Whh.shape)
            dbh = torch.zeros(bh.shape)
            dWxh = torch.zeros(Wxh.shape)

            #Start working in reverse order and collect derivative at each step
            for i in range(seq_len)[::-1]:
            #for i in range(10)[::-1]:


                #position of 1 in target vector (y), implies 100% probability of that character
                #####y_idx = torch.argmax(targets[i])  #position of 1 in target vector (y), implies 100% probability of that character


                y_idx = cur_tar_seq[i]
                #select the probability at same position and subtract 1 (y_hat - y)    
                ps[i][y_idx] -= 1 


                #For Why and by derivative with respect to error is sum of derivatives at each step or step-error
                dWhy_step_i=ps[i] @ (hs[i]).t()      #(y_hat - y) @ hs.T - for each step

                dby = dby + ps[i]                    #ps[i] represents (y_hat - y) as computed above

                dWhy = dWhy + dWhy_step_i            #sum over each step


                #Computing delta for Whx, Whh and bh invloves computing step-error
                #Followed by finding differential of parameters each step-error over the whole sequnce
                #computing and updating delta_z helps to keep track of and sum up throught out the sequence

                #delta with respect to a single step
                delta_recent = (Why.t()@ ps[i]) * (1-hs[i]**2)

                #taking all the way back to first step
                for j in range(i+1)[::-1]:

                    dWhh_ij= delta_recent @ hs[j-1].t()
                    dbh = dbh + delta_recent

                    dWhh = dWhh + dWhh_ij

                    dWxh_ij= delta_recent @ xs[j].t()
                    dWxh = dWxh + dWxh_ij

                    #update delta_z such that it goes back one step all while computing the derivative with repect
                    #to same step-error
                    delta_recent = (Whh.t() @ delta_recent) * (1-hs[j-1]**2)




            mWhy += dWhy * dWhy
            mWxh += dWxh * dWxh
            mWhh += dWhh * dWhh
            mbh += dbh * dbh
            mby += dby * dby

            Why += - l_r * dWhy/np.sqrt(mWhy +eps)
            Whh +=  - l_r * dWhh/np.sqrt(mWhh +eps)
            Wxh +=  - l_r * dWxh/np.sqrt(mWxh +eps)
            by +=  - l_r * dby/np.sqrt(mby +eps)
            bh +=  - l_r * dbh/np.sqrt(mbh +eps)


    #         Why = Why - lr * dWhy
    #         Whh = Whh - lr * dWhh
    #         Wxh = Wxh - lr * dWxh
    #         by = by - lr *dby
    #         bh = bh - lr *dbh

        losses.append(loss.item())
        if k % 2 ==0 :
            print(k, loss.item())
    return (losses)
            
        #adjust the parameters


In [115]:
Wxh2, Whh2, Why2, bh2, by2 = init_weights(h_size, vocab_size)
input_seq, target_seq = get_tensors_alt(all_names[:950], 12)
losses2 = rnn_net(Wxh2, Whh2, Why2, bh2, by2, input_seq, target_seq, 50)
gen_names(Wxh2, Whh2, Why2, bh2, by2) #loss 2.27

0 6.120089054107666
2 2.6177220344543457
4 2.3097617626190186
6 2.222156047821045
8 2.191105365753174
10 2.1795356273651123
12 2.176025629043579
14 2.1761364936828613
16 2.17785906791687
18 2.1802215576171875
20 2.1827383041381836
22 2.1851656436920166
24 2.187401533126831
26 2.1894168853759766
28 2.1912121772766113
30 2.1928040981292725
32 2.1942245960235596
34 2.1954948902130127
36 2.196655511856079
38 2.1977169513702393
40 2.198709487915039
42 2.199643611907959
44 2.2005410194396973
46 2.201413869857788
48 2.202275037765503


['alio',
 'aadarsh',
 'aniu',
 'amita',
 'alina',
 'ashetm',
 'abink',
 'abhiru',
 'aneesh',
 'avishek',
 'aalek',
 'apekshya',
 'amlen',
 'anmol',
 'aeenadh',
 'aamil',
 'arujidr',
 'ajin',
 'alina',
 'abheep',
 'arida',
 'asekh',
 'aites',
 'albina',
 'aneela',
 'avisekh',
 'areenca',
 'askula',
 'anuraag',
 'asbha',
 'adhan',
 'arbina',
 'andrew',
 'arbina',
 'alina',
 'anika',
 'ashuka',
 'anuruda',
 'abhimar',
 'anur',
 'abhila',
 'apshaa',
 'aworakh',
 'anteea',
 'aditi',
 'ashma',
 'anyula',
 'ayusha',
 'aplish',
 'ajendr']

In [124]:
#plt.plot(losses)

In [102]:

gen_names_new = gen_names(Wxh2, Whh2, Why2, bh2, by2)
', '.join(w.capitalize() for w in gen_names_new)#.split(' '))

'Anjeep, Anbooda, Abinish, Ashma, Avilesh, Ajilekh, Avineek, Areshagran, Ashriti, Akaran, Anjaar, Awachmi, Arja, Arina, Apeeth, Ankulan, Anjalee, Anudekran, Aashuraa, Adeenaaz, Abinash, Anubhag, Alooh, Avicekh, Alech, Aplesh, Askar, Anikabh, Aratumu, Amita, Asyab, Allee, Arbilae, Ashuta, Anishab, Akshee, Abhishwa, Ankesh, Arrjetta, Araji, Agrish, Avismar, Abadees, Andeekh, Anuton, Asidea, Arbha, Anmol, Anbusana, Aayjeshra'

In [114]:
gen_names_new = gen_names(Wxh2, Whh2, Why2, bh2, by2)
', '.join(w.capitalize() for w in gen_names_new)

'Araola, Aanim, Aalish, Aarjal, Amreth, Aisha, Adeeti, Angmi, Ambha, Ankeek, Ajay, Akhish, Arnang, Abhila, Aasha, Aranin, Akmit, Arunamr, Ayooh, Anumb, Arbina, Abiya, Amar, Avicesh, Ayushi, Anir, Adodra, Anidu, Awirish, Apeklacay, Anar, Aamish, Ajeet, Aaresh, Anurutpa, Anujila, Abekh, Apima, Anurabgne, Awadhkhstoi, Anureep, Arif, Aksha, Apec, Avichan, Anurudr, Arajiyu, Alone, Aakshana, Aajadr'